### P1

#### fortran-p1.f90

``` fortran
program P1

    implicit none
   
    integer :: i, j, fu
    ! character(len=*), parameter :: OUT_FILE = 'data-p1.txt' ! Output file.
    character(len=*), parameter :: PLT_SAVE = 'plot-p1.plt'   ! Save
    
    double precision :: x_vec(100) = 0
    double precision :: y_vec(100) = 0
    double precision :: z_vec(100, 100) = 0
    

    ! Computing the sin(2x) fucntion
    do i = 1, 100
        x_vec(i) = dble(i) / 20.0
        y_vec(i) = sin(2*x_vec(i))
    end do

    ! print *, x_vec
    
    
    ! Writing results in ".txt" files
    open (action='write', file="data-p1-sin.txt", newunit=fu, status='replace')
    do i = 1, 100
        write (fu, '(F10.5 F10.5)') x_vec(i), y_vec(i)
    end do
    close(fu)

    ! Now we create a 3D plot of tanh(x*y) with nestded loops
    open (action='write', file="data-p1-surface.txt", newunit=fu, status='replace')
    do i = 1, 100
        x_vec(i) = dble(i) / 20.0 - 2.5
        do j = 1, 100
            y_vec(j) = dble(j) / 20.0 - 2.5
            z_vec(i,j) = tanh(x_vec(i)*y_vec(j))

            write (fu, '(F10.5 F10.5 F10.5)') x_vec(i), y_vec(j), z_vec(i,j)
        end do
        write (fu, *) ""
    end do
    close(fu)


    print *, "Ploting started"
    call execute_command_line('gnuplot -p ' // PLT_SAVE)    ! Plot and save
    
    
    print *, "The program is finished"
end program P1
```

#### plot-p1.plt (for gnuplot)

``` gnuplot
# plot-save.plt
set terminal pngcairo size 8000, 8000 fontscale 10 pointscale 1
set size square
set output "sin-plot.png"
set xrange[0:5]
set yrange[-1.5:1.5]
set title "y = Sin(2x)"

plot "data-p1-sin.txt" with linespoints pt 30 pointsize 10 title ""

clear

set xrange[-2.5:2.5]
set yrange[-2.5:2.5]
set output "surface-plot.png"
set title "z = Tanh(x*y)"
set hidden3d

splot "data-p1-surface.txt" with lines lc rgb '#b90046' lw 1.5 title ""



```

#### Results

![Alt text](fortran-code/p1/sin-plot.png)

![Alt text](fortran-code/p1/surface-plot.png)

### P2

#### fortran-p2.f90

``` fortran
program P2

    implicit none
   
    integer :: i, fu
    character(len=*), parameter :: PLT_SAVE = 'plot-p2.plt'     ! Save
    integer, parameter :: dp = selected_real_kind(15, 307)      ! Double-precision type
    
    integer, parameter :: N = 1000                              ! Dimension of the matrix
    real(dp) :: A_real(N,N), A_img(N,N)
    complex(dp) :: A(N,N), A_t(N,N)
    complex(dp) :: H_sum(N,N), H_mul(N,N)
    real(dp) :: H_sum_eigen(N), H_mul_eigen(N)
    


    call random_seed()                    ! Setting the random seed
    call random_number(A_real)            ! Random real part
    call random_number(A_img)             ! Random imagenary part

    ! Creating A and adjoint of it
    A = A_real + A_img*complex(0.0, 1.0)
    A_t = conjg(transpose(A))

    ! Making Hermitian matrices 
    H_sum = A + A_t                         
    H_mul = matmul(A, A_t)
    
    ! Computing the eigenvalues 
    H_sum_eigen = calc_eigen(H_sum, N)
    H_mul_eigen = calc_eigen(H_mul, N)
    
    ! Writing results in ".txt" files
    open (action='write', file="data-p2-sum.txt", newunit=fu, status='replace')
    do i = 1, N-1
        write (fu, '(F10.5)') H_sum_eigen(i)
    end do
    close(fu)

    open (action='write', file="data-p2-mul.txt", newunit=fu, status='replace')
    do i = 1, N-1
        write (fu, '(F10.5)') H_mul_eigen(i)
    end do
    close(fu)



    print *, "Ploting started"
    call execute_command_line('gnuplot -p ' // PLT_SAVE)    ! Plot and save
    
    
    print *, "The program is finished"


    contains

    function  calc_eigen(mat, N_mat)     ! Calculates eigenvalues of hermitian matrices

        ! This functions uses the ZHEEV subroutine from LAPACK lib.

        implicit none

        integer :: N_mat
        integer :: LWORK
        complex(dp) :: mat(N_mat, N_mat)
        complex(dp) :: mat_copy(N_mat, N_mat)
        real(dp) :: eigen_out(N_mat), calc_eigen(N_mat)
        complex(dp) :: WORK(6*N_mat)
        real(dp) :: RWORK(3*N_mat - 2)
        integer :: info

        LWORK = 6*N_mat

        mat_copy = mat
        call ZHEEV('N', 'U', N_mat, mat_copy, N_mat, eigen_out, WORK, LWORK, RWORK, info)

        print *, "ZHEEV info:", info, "\n"
        calc_eigen = eigen_out

    end function calc_eigen


end program P2
```

#### plot-p2.plt

```

# plot-save.plt
set terminal pngcairo size 2000, 2000 fontscale 4
set style fill solid border -1
set key off


binwidth=1.5
bin(x,width)=width*floor(x/width)


set title "A * A'  Eigenvalues"
set output "hist-mul.png"
plot 'data-p2-mul.txt' using (bin($1,binwidth)):(1.0) smooth freq with boxes

clear

set title "A + A'  Eigenvalues"
set output "hist-sum.png"
plot 'data-p2-sum.txt' using (bin($1,binwidth)):(1.0) smooth freq with boxes



```

#### Results

![Alt text](fortran-code/p2/hist-sum.png)

![Alt text](fortran-code/p2/hist-mul.png)